In [ ]:
from zipfile import ZipFile
  
zip_file = "/content/drive/MyDrive/FER2013/archive.zip"

# opening the zip file in READ mode
with ZipFile(zip_file, 'r') as zip:
    # extracting all the files
    print('Extracting files...')
    zip.extractall("/content/drive/MyDrive/FER2013/")
    print('Done!')

Extracting files...
Done!


In [ ]:
import os
import numpy as np

# fetching path for each image in train and test directories
def loadData(path):
  data = []
  labels = []
  for folder in os.listdir(path):
    for img in os.listdir(path + "/" + folder):
      data.append((path + "/" + folder + "/" + img))
      labels.append(folder)
    print("Fetching image path from ",path + "/" + folder)
  return np.array(data), np.array(labels)

train_path, train_labels = loadData("/content/drive/MyDrive/FER2013/train")
print("\n")
test_path, test_labels = loadData("/content/drive/MyDrive/FER2013/test")
print("\n")
print("Training Images : ",train_path.shape)
print("Training Labels : ",train_labels.shape)
print("Testing  Images : ",test_path.shape)
print("Testing  Labels : ",test_labels.shape)

Fetching image path from  /content/drive/MyDrive/FER2013/train/angry
Fetching image path from  /content/drive/MyDrive/FER2013/train/disgust
Fetching image path from  /content/drive/MyDrive/FER2013/train/fear
Fetching image path from  /content/drive/MyDrive/FER2013/train/happy
Fetching image path from  /content/drive/MyDrive/FER2013/train/neutral
Fetching image path from  /content/drive/MyDrive/FER2013/train/sad
Fetching image path from  /content/drive/MyDrive/FER2013/train/surprise


Fetching image path from  /content/drive/MyDrive/FER2013/test/angry
Fetching image path from  /content/drive/MyDrive/FER2013/test/disgust
Fetching image path from  /content/drive/MyDrive/FER2013/test/fear
Fetching image path from  /content/drive/MyDrive/FER2013/test/happy
Fetching image path from  /content/drive/MyDrive/FER2013/test/neutral
Fetching image path from  /content/drive/MyDrive/FER2013/test/sad
Fetching image path from  /content/drive/MyDrive/FER2013/test/surprise


Training Images :  (28709,)
T

In [ ]:
import cv2

# using train and test image path loaing image as array into one variable
def resizeImages(images):

  X = np.ndarray((images.shape[0],224,224,3), dtype=np.uint8)
  Y = np.ndarray((images.shape[0],48,48,3), dtype=np.uint8)

  for i, img in enumerate(images):
    # fetching image from path
    imgRead = cv2.imread(img, cv2.IMREAD_COLOR)

    # resizing image from 48x48 pixels to 224x224 pixels
    X[i, :] = cv2.resize(imgRead, (224, 224), interpolation=cv2.INTER_CUBIC)
    Y[i, :] = imgRead

    if i%500 == 0:
      print("Processed Images = {}".format(i))
    if i == (images.shape[0]-1):
      print("Processed Images = {}".format(i+1))

  return X, Y

In [ ]:
print("Processing train data .... \n")
train_data, train_data48 = resizeImages(train_path)

Processing train data .... 

Processed Images = 0
Processed Images = 500
Processed Images = 1000
Processed Images = 1500
Processed Images = 2000
Processed Images = 2500
Processed Images = 3000
Processed Images = 3500
Processed Images = 4000
Processed Images = 4500
Processed Images = 5000
Processed Images = 5500
Processed Images = 6000
Processed Images = 6500
Processed Images = 7000
Processed Images = 7500
Processed Images = 8000
Processed Images = 8500
Processed Images = 9000
Processed Images = 9500
Processed Images = 10000
Processed Images = 10500
Processed Images = 11000
Processed Images = 11500
Processed Images = 12000
Processed Images = 12500
Processed Images = 13000
Processed Images = 13500
Processed Images = 14000
Processed Images = 14500
Processed Images = 15000
Processed Images = 15500
Processed Images = 16000
Processed Images = 16500
Processed Images = 17000
Processed Images = 17500
Processed Images = 18000
Processed Images = 18500
Processed Images = 19000
Processed Images = 1

In [ ]:
# suffuling train data
shuffler = np.random.permutation(len(train_data))
train_data = train_data[shuffler]
train_data48 = train_data48[shuffler]
train_labels = train_labels[shuffler]

# one hot train encode labels
unique, inverse = np.unique(train_labels, return_inverse=True)
train_labels = np.eye(unique.shape[0])[inverse]
print("Training Data    : ",train_data.shape)
print("Training Data(48): ",train_data48.shape)
print("Training Labels  : ",train_labels.shape)

# Saving train images and labels into .npy file
np.save('/content/drive/MyDrive/FER2013/train_data.npy', train_data)
np.save('/content/drive/MyDrive/FER2013/train_data48.npy', train_data48)
np.save('/content/drive/MyDrive/FER2013/train_labels.npy', train_labels)

del train_data48

Training Data    :  (28709, 224, 224, 3)
Training Data(48):  (28709, 48, 48, 3)
Training Labels  :  (28709, 7)


In [ ]:
def sobelEdgeDetection(img):
  scale = 1
  delta = 0
  ddepth = cv2.CV_16S

  grad_x = cv2.Sobel(img, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
  grad_y = cv2.Sobel(img, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)

  abs_grad_x = cv2.convertScaleAbs(grad_x)
  abs_grad_y = cv2.convertScaleAbs(grad_y)
  
  image_edges = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
  return image_edges

In [ ]:
train_canny_edges = np.ndarray((train_data.shape[0],224,224), dtype=np.uint8)
train_sobel_edges = np.ndarray((train_data.shape[0],224,224,3), dtype=np.uint8)

for i in range(len(train_data)):
  
  # applying canny edge detector
  train_canny_edges[i, :] = cv2.Canny(train_data[i],70,100)
  
  # applying sobel edge detector
  train_sobel_edges[i, :] = sobelEdgeDetection(train_data[i])

In [ ]:
# Saving filtered train images into .npy file
np.save("/content/drive/MyDrive/FER2013/sobel_train.npy",train_sobel_edges)
np.save("/content/drive/MyDrive/FER2013/canny_train.npy",train_canny_edges)

del train_data
del train_sobel_edges
del train_canny_edges

In [ ]:
print("Processing test data .... \n")
test_data, test_data48 = resizeImages(test_path)

Processing test data .... 

Processed Images = 0
Processed Images = 500
Processed Images = 1000
Processed Images = 1500
Processed Images = 2000
Processed Images = 2500
Processed Images = 3000
Processed Images = 3500
Processed Images = 4000
Processed Images = 4500
Processed Images = 5000
Processed Images = 5500
Processed Images = 6000
Processed Images = 6500
Processed Images = 7000
Processed Images = 7080


In [ ]:
# suffuling test data
shuffler = np.random.permutation(len(test_data))
test_data = test_data[shuffler]
test_data48 = test_data48[shuffler]
test_labels = test_labels[shuffler]

# one hot test encode labels
unique, inverse = np.unique(test_labels, return_inverse=True)
test_labels = np.eye(unique.shape[0])[inverse]
print("Testing  Data    : ",test_data.shape)
print("Testing  Data(48): ",test_data48.shape)
print("Testing  Labels  : ",test_labels.shape)

Testing  Data    :  (7080, 224, 224, 3)
Testing  Data(48):  (7080, 48, 48, 3)
Testing  Labels  :  (7080, 7)


In [ ]:
# Saving test images and labels into .npy file
np.save('/content/drive/MyDrive/FER2013/test_data.npy', test_data)
np.save('/content/drive/MyDrive/FER2013/test_data48.npy', test_data48)
np.save('/content/drive/MyDrive/FER2013/test_labels.npy', test_labels)

del test_data48

In [ ]:
test_canny_edges = np.ndarray((test_data.shape[0],224,224), dtype=np.uint8)
test_sobel_edges = np.ndarray((test_data.shape[0],224,224,3), dtype=np.uint8)

for i in range(len(test_data)):
  
  # applying canny edge detector
  test_canny_edges[i, :] = cv2.Canny(test_data[i],70,100)
  
  # applying sobel edge detector
  test_sobel_edges[i, :] = sobelEdgeDetection(test_data[i])

In [ ]:
np.save("/content/drive/MyDrive/FER2013/sobel_test.npy",test_sobel_edges)
np.save("/content/drive/MyDrive/FER2013/canny_test.npy",test_canny_edges)

del test_data
del test_sobel_edges
del test_canny_edges